<a href="https://colab.research.google.com/github/Utpal-Mishra/Bidnamics/blob/main/Bidnamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [93]:
path = "/content/drive/MyDrive/Jobs/Binamics/hack_day_data_train.csv"
data = pd.read_csv(path)
print('Data Shape: {}'.format(data.shape))

data.head()

Data Shape: (6415, 11)


,offerId,title,description,itemGroupId,price,productTypes,sizes,color,brand,clicks,conversions
0,shopify_gb_7124848804012_40995451699372,Limited Edition Valentine's Phone Case - Leopa...,Calling all leopard loving ladies! We've got t...,7.124849e+12,"{'value': '25.00', 'currency': 'GBP'}",['Tough Phone Case'],['Samsung Galaxy S21 Plus'],NaN,NaN,0,0.0
1,shopify_gb_4421341118562_31614566236258,"Personalised Ipad Case - Starry Night, Ipad Mi...",This iPad Case has put stars in our eyes! Let ...,4.421341e+12,"{'value': '35.00', 'currency': 'GBP'}",['Personalised Ipad Case'],['Ipad Mini 4'],black,NaN,2,0.0
2,shopify_gb_4468302676066_31826267537506,"Tough Phone Case - Oranges, Samsung Galaxy S6 ...",Orange you glad we created a JUICY Phone Case ...,4.468303e+12,"{'value': '25.00', 'currency': 'GBP'}",['Tough Phone Case'],['Samsung Galaxy S6 Edge'],Nude,NaN,0,0.0
3,shopify_gb_3945047162978_29455211331682,"Personalised Phone Case - Grey & Black Marble,...",Who knew a phone case could look so…classy?! W...,3.945047e+12,"{'value': '19.00', 'currency': 'GBP'}",['Personalised Phone Case'],['iPhone X/XS'],NaN,Coconut Lane,5,0.0
4,shopify_gb_7006203674796_40653202915500,"Tough Phone Case - Brown Swirls, Samsung Galax...",Nothing screams Autumn more than brown swirls ...,7.006204e+12,"{'value': '25.00', 'currency': 'GBP'}",['Tough Phone Case'],['Samsung Galaxy S7'],NaN,NaN,3,0.0


## DATA CLEANING

### Fetching Price

In [94]:
for i in range(len(data)):
  data.price[i] = float(data.price[i].split("'")[3])

data.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,offerId,title,description,itemGroupId,price,productTypes,sizes,color,brand,clicks,conversions
6410,shopify_gb_4357137989730_31987515883618,"Candy Hearts Phone Case, iPhone 12 Mini",Keep it cute wherever you go with our super sw...,4.357138e+12,14.0,['Phone Case'],['iPhone 12 Mini'],pink,Coconut Lane,66,2.0
6411,shopify_gb_4421311037538_37851058765996,"Personalised Ipad Case - House Plants, Ipad 10...",Personalisation and cute prints your thing? Yo...,4.421311e+12,35.0,['Personalised Ipad Case'],['Ipad 10.2 (2020/8th Gen)'],pink,NaN,35,2.0
6412,shopify_gb_6832981344428_40137802186924,"Stay Wild Set, UK 14 / UK 6",Meeow! Looking for your new summer staple outf...,6.832981e+12,65.0,['Sets'],['UK 6'],NaN,NaN,2,0.0
6413,shopify_gb_7104510558380_40916107231404,"Tough Phone Case - Nude Snake, Samsung Galaxy ...",Introducing our Nude Snake Tough Phone Case - ...,7.104511e+12,25.0,['Tough Phone Case'],['Samsung Galaxy S9 Plus'],NaN,NaN,0,0.0
6414,shopify_gb_7124847919276_40995446685868,Limited Edition Valentine's Phone Case - Love ...,For this case we've collaborated with the love...,7.124848e+12,19.0,['Tough Phone Case'],['iPhone 7/8'],NaN,NaN,0,0.0


### Advertising Metrics

In [95]:
conversationRate = []
averageCostPerClick = []

for i in range(len(data)):
  if data.clicks[i] == 0:
    conversationRate.append(data.clicks[i])
    averageCostPerClick.append(data.clicks[i])
  else:
    conversationRate.append(data.conversions[i]/data.clicks[i])
    averageCostPerClick.append(data.price[i]/float(data.clicks[i]))

data['conversationRate'] = conversationRate
data['averageCostPerClick'] = averageCostPerClick

### Dealing with NULL Values

In [96]:
for i in range(len(data)):
  try:
    data.productTypes[i] = data.productTypes[i].split("'")[1]
  except: 
    data.productTypes[i] = 'Unknown'

for i in range(len(data)):
  try:
    data.sizes[i] = data.sizes[i].split("'")[1]
  except: 
    data.sizes[i] = 'Unknown'

data.color.fillna("Unknown")
data.brand.fillna("Unknown")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying 

0            Unknown
1            Unknown
2            Unknown
3       Coconut Lane
4            Unknown
            ...     
6410    Coconut Lane
6411         Unknown
6412         Unknown
6413         Unknown
6414         Unknown
Name: brand, Length: 6415, dtype: object

### Branding

In [108]:
brand = []

for i in range(len(data.title)):
  if 'samsung' in data.title[i].lower():
    brand.append('Samsung')
  elif 'iphone' in data.title[i].lower() or 'ipad' in data.title[i].lower() or 'notebook' in data.title[i].lower() or 'airtag' in data.title[i].lower() or 'airpod' in data.title[i].lower():
    brand.append('Apple')
  elif 'fitbit' in data.title[i].lower():
    brand.append('Fitbit')
  elif 'huawei' in data.title[i].lower():
    brand.append('Huawei')
  elif 'bag' in data.title[i].lower():
    brand.append('Unknwon Bag Brand')
  elif 'ring' in data.title[i].lower():
    brand.append('Unknwon Ring Brand')
  elif 'face covering' in data.title[i].lower():
    brand.append('Unknwon Face Covering Brand')
  elif 'tag' in data.title[i].lower():
    brand.append('Unknwon Face Covering Brand')
  elif 'phone case' in data.title[i].lower():
    brand.append('Unknwon Phone Case Brand')
  else: brand.append(data.productTypes[i])

data['brandName'] = brand
data.head()

,offerId,title,description,itemGroupId,price,productTypes,sizes,color,brand,clicks,conversions,conversationRate,averageCostPerClick,brandName
0,shopify_gb_7124848804012_40995451699372,Limited Edition Valentine's Phone Case - Leopa...,Calling all leopard loving ladies! We've got t...,7.124849e+12,25.0,Tough Phone Case,Samsung Galaxy S21 Plus,NaN,NaN,0,0.0,0.0,0.000000,Samsung
1,shopify_gb_4421341118562_31614566236258,"Personalised Ipad Case - Starry Night, Ipad Mi...",This iPad Case has put stars in our eyes! Let ...,4.421341e+12,35.0,Personalised Ipad Case,Ipad Mini 4,black,NaN,2,0.0,0.0,17.500000,Apple
2,shopify_gb_4468302676066_31826267537506,"Tough Phone Case - Oranges, Samsung Galaxy S6 ...",Orange you glad we created a JUICY Phone Case ...,4.468303e+12,25.0,Tough Phone Case,Samsung Galaxy S6 Edge,Nude,NaN,0,0.0,0.0,0.000000,Samsung
3,shopify_gb_3945047162978_29455211331682,"Personalised Phone Case - Grey & Black Marble,...",Who knew a phone case could look so…classy?! W...,3.945047e+12,19.0,Personalised Phone Case,iPhone X/XS,NaN,Coconut Lane,5,0.0,0.0,3.800000,Apple
4,shopify_gb_7006203674796_40653202915500,"Tough Phone Case - Brown Swirls, Samsung Galax...",Nothing screams Autumn more than brown swirls ...,7.006204e+12,25.0,Tough Phone Case,Samsung Galaxy S7,NaN,NaN,3,0.0,0.0,8.333333,Samsung


## Interpretations

### Frequency of Different Products Bought

In [110]:
productCount = pd.DataFrame(data.productTypes.value_counts()).reset_index().sort_values('index')
productCount.columns = ['productName', 'freq']
fig = go.Figure(data=[go.Pie(labels=productCount["productName"], values=productCount.freq, hole=.5)])
fig.update_layout(height=1000, width=1400, title_text='Frequency of Different Products Bought') 
fig.show()

### Total Clicks

In [99]:
totalClicks = pd.DataFrame(data.groupby(['productTypes'])['clicks'].sum()).reset_index()
totalClicks.head()

,productTypes,clicks
0,AirTag Cases,2699
1,Airpod Cases,15457
2,Apple Watch Straps,168156
3,Beauty Balms,20
4,Bracelet,6


In [100]:
fig = px.bar(totalClicks, y=totalClicks.productTypes, x=totalClicks.clicks, color=totalClicks.clicks)
fig.update_xaxes(title_text = "Product Types", rangeslider_visible=False, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(title_text = 'Clicks', showline=True, linewidth=2, linecolor='black', mirror=True)
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(height=1000, width=1400, title_text='Total Clicks for Product Types') 
fig.show()

### Conversion Rate Comparision

In [101]:
conversionRate = pd.DataFrame(data.groupby(['productTypes'])['conversationRate'].sum()).reset_index()
conversionRate.head()

,productTypes,conversationRate
0,AirTag Cases,0.768465
1,Airpod Cases,3.624271
2,Apple Watch Straps,4.822729
3,Beauty Balms,0.000000
4,Bracelet,0.000000


In [102]:
fig = px.bar(conversionRate, y=conversionRate.productTypes, x=conversionRate.conversationRate, color=conversionRate.conversationRate)
fig.update_xaxes(title_text = "Conversion Rate", rangeslider_visible=False, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(title_text = 'Product Types', showline=True, linewidth=2, linecolor='black', mirror=True)
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(height=1000, width=1400, title_text='Conversion Rate for Product Types') 
fig.show()

### Conversion Rate Comparision

In [103]:
averageCostPerClick = pd.DataFrame(data.groupby(['productTypes'])['averageCostPerClick'].sum()).reset_index()
averageCostPerClick.head()

,productTypes,averageCostPerClick
0,AirTag Cases,3.913488
1,Airpod Cases,120.627182
2,Apple Watch Straps,364.379177
3,Beauty Balms,2.352941
4,Bracelet,5.333333


In [104]:
fig = px.bar(averageCostPerClick, y=averageCostPerClick.productTypes, x=averageCostPerClick.averageCostPerClick, color=averageCostPerClick.averageCostPerClick)
fig.update_xaxes(title_text = "Average Cost Per Click", rangeslider_visible=False, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(title_text = 'Product Type', showline=True, linewidth=2, linecolor='black', mirror=True)
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(height=1000, width=1400, title_text='Average Cost Per Click for Product Types') 
fig.show()

### Conversion Rate vs Average Cost Per Click Comparision

In [105]:
compare = pd.DataFrame(data.groupby(['productTypes'])['clicks', 'conversationRate', 'averageCostPerClick'].sum()).reset_index().sort_values('conversationRate')
compare.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,productTypes,clicks,conversationRate,averageCostPerClick
39,Sets,7,0.0,184.166667
44,Travel Set,1,0.0,20.000000
9,Charm,29,0.0,15.312500
3,Beauty Balms,20,0.0,2.352941
4,Bracelet,6,0.0,5.333333


In [106]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=compare.productTypes, y=compare.conversationRate, mode='lines', name='Conversation Rate', line=dict(width = 2)))
fig.add_trace(go.Scatter(x=compare.productTypes, y=compare.averageCostPerClick, mode='lines+markers', name='Average Cost Per Click'))
#fig.add_trace(go.Scatter(x=compare.productTypes, y=compare.clicks, mode='markers', name='Conversation Rate', line=dict(width = 3)))
fig.update_xaxes(title_text = "Product Type", rangeslider_visible=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(height=700, width=1400, title_text='Conversation Rate vs Average Cost Per Click For Product Types')
fig.show()

### Frequency of Different Brand Products Bought

In [107]:
brandCount = pd.DataFrame(data.brandName.value_counts()).reset_index().sort_values('index')
brandCount.columns = ['brandName', 'freq']
fig = go.Figure(data=[go.Pie(labels=brandCount["brandName"], values=brandCount.freq, hole=.5)])
fig.update_layout(height=1000, width=1400, title_text='Frequency of Different Brand Products Bought') 
fig.show()

## Sentimental Analysis

In [17]:
from tqdm import tqdm
import re
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
from textblob import TextBlob
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
sentiment = pd.DataFrame(data[['productTypes', 'description']])
sentiment

,productTypes,description
0,Tough Phone Case,Calling all leopard loving ladies! We've got t...
1,Personalised Ipad Case,This iPad Case has put stars in our eyes! Let ...
2,Tough Phone Case,Orange you glad we created a JUICY Phone Case ...
3,Personalised Phone Case,Who knew a phone case could look so…classy?! W...
4,Tough Phone Case,Nothing screams Autumn more than brown swirls ...
...,...,...
6410,Phone Case,Keep it cute wherever you go with our super sw...
6411,Personalised Ipad Case,Personalisation and cute prints your thing? Yo...
6412,Sets,Meeow! Looking for your new summer staple outf...
6413,Tough Phone Case,Introducing our Nude Snake Tough Phone Case - ...


In [19]:
corpus=[]
for i in tqdm(range(data.shape[0])):
  review = re.sub('[^a-zA-z]',' ', str(data['description'][i]))
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review) 

100%|██████████| 6415/6415 [03:00<00:00, 35.59it/s]


In [20]:
analyser = SentimentIntensityAnalyzer()

compound = []
neg = []
neu = []
pos = []

for i in tqdm(range(len(corpus))):
  compound.append(analyser.polarity_scores(corpus[i])['compound'])
  neg.append(analyser.polarity_scores(corpus[i])['neg'])
  neu.append(analyser.polarity_scores(corpus[i])['neu'])
  pos.append(analyser.polarity_scores(corpus[i])['pos'])

sentiment['compound'] = compound
sentiment['neg'] = neg
sentiment['neu'] = neu
sentiment['pos'] = pos

100%|██████████| 6415/6415 [00:49<00:00, 130.06it/s]


In [21]:
sentiment.groupby(['productTypes']).mean().reset_index()

,productTypes,compound,neg,neu,pos
0,AirTag Cases,0.925473,0.090636,0.642909,0.266636
1,Airpod Cases,0.813443,0.062968,0.682079,0.254841
2,Apple Watch Straps,0.915236,0.033992,0.771902,0.194074
3,Beauty Balms,0.963050,0.000000,0.707000,0.293000
4,Bracelet,0.891000,0.000000,0.808000,0.192000
5,Bundle,0.413665,0.046500,0.866607,0.086840
6,Cami Tops,0.805767,0.000000,0.800000,0.200000
7,Candles,0.709804,0.035577,0.725154,0.239308
8,Card Holders,0.651171,0.060286,0.739571,0.200000
9,Charm,0.943709,0.027156,0.705313,0.267500
